# Test new nf run linking

In [ ]:
!lamin load nextflow-mcmicro

In [ ]:
import lamindb as ln
from subprocess import getoutput

In [ ]:
!nextflow run https://github.com/labsyspharm/mcmicro --in exemplar-001 --start-at illumination --stop-at registration -with-report execution_report.html

In [ ]:
# !python register_mcmicro_run_link.py

In [ ]:
transform = ln.Transform(
    name="mcmicro",
    version="1.0.0",
    type="pipeline",
    reference="https://github.com/labsyspharm/mcmicro",
)
run = ln.track(transform=transform)

ln.settings.sync_git_repo = "https://github.com/laminlabs/nextflow-lamin-usecases"

In [ ]:
ulabel = ln.ULabel(name="nextflow").save()
transform.ulabels.add(ulabel)

In [ ]:
import yaml
from lamindb.core._run_context import parse_and_link_params

with open("exemplar-001/qc/params.yml") as params_file:
    params = yaml.safe_load(params_file)

parse_and_link_params(run, {k: v for k, v in params.items() if v is not None})

In [ ]:
run.param_values.all()

In [ ]:
mcmicro_input = ln.Artifact.filter(key__startswith="exemplar-001")
run.input_artifacts.set(mcmicro_input)

In [ ]:
output = ln.Artifact(
    "exemplar-001/registration/exemplar-001.ome.tif", description="mcmicro"
)
output.save()
run.output_artifacts.set([output])

In [ ]:
report = ln.Artifact("execution_report.html").save()
run.report = report
run.save()

In [ ]:
nextflow_id = getoutput(f"nextflow log | awk '/{run.id}/{{print $8}}'")
run.reference = nextflow_id
run.reference_type = "nextflow_id"
run.save()

In [ ]:
run.transform

In [ ]:
%tb
ln.finish()